In [2]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

c:\programacao\automacao\Llama\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
llm = Groq(model="llama3-70b-8192", 
           api_key=os.environ.get("GROQ_API_KEY"))

In [4]:
def calcular_imposto_renda(rendimento:float) -> str:
    """
    calcular o imposto de renda com base no rendimento atual.
    Args:
        rendimento(float) : Redndimetno anual do individuo.

    Returns:
        str: o valor do imposto devido com base no rendimento
    """
    if rendimento <=2000:
        return "Você esta isento de pagar imposto de renda"
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    elif 5000 < rendimento <= 10000: 
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"

In [5]:
from llama_index.core.tools import FunctionTool

In [6]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name= "calcular imposto de Renda",
    description=(
        "calcula o imposto de renda com base no rendimento atual"
        "Argumento: redimento (float)"
        "retorna o valor do imposto devido de acordo com faixa de rendimento")
)

In [7]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [8]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [9]:
from llama_index.core.agent import AgentRunner

In [10]:
agent_imposto = AgentRunner(agent_worker_imposto)

C:\Users\arthu\AppData\Local\Temp\ipykernel_11324\1872460396.py:1: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  agent_imposto = AgentRunner(agent_worker_imposto)


In [11]:
response = agent_imposto.chat("""
Qual é o imposto de renda devido de uma pessoa com rendimento anual de 7500 reais?
""")

Added user message to memory: 
Qual é o imposto de renda devido de uma pessoa com rendimento anual de 7500 reais?

=== Calling Function ===
Calling function: calcular imposto de Renda with args: {"rendimento": 7500}
=== Function Output ===
O imposto devido é de R$ 675.00, base em um rendimento de R$ 7500.00
=== LLM Response ===
O imposto devido é de R$ 675.00, base em um rendimento de R$ 7500.00


In [12]:
import arxiv

def consulta_artigos(titulo: str) -> str:
    """Consulta os artigos na base de dados ArXiv e retorna os resultados formatados"""
    busca = arxiv.Search(
        query = titulo,
        max_results = 5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = [
        f"Título: {artigo.title} \n"
        f"Categoria: {artigo.primary_category} \n"
        f"Link: {artigo.entry_id} \n"
        for artigo in busca.results()
    ]

    return "\n\n".join(resultados)


In [13]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [14]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [15]:
agent = AgentRunner(agent_worker)
response= agent.chat("Me retorne artigos sobre langChain na educação")

Added user message to memory: Me retorne artigos sobre langChain na educação


C:\Users\arthu\AppData\Local\Temp\ipykernel_11324\3122400025.py:1: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  agent = AgentRunner(agent_worker)


=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "langChain educa\u00e7\u00e3o"}


C:\Users\arthu\AppData\Local\Temp\ipykernel_11324\3843034253.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for artigo in busca.results()


=== Function Output ===
Título: From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integrated Web Application? 
Categoria: cs.CR 
Link: http://arxiv.org/abs/2308.01990v4 


Título: Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations 
Categoria: cs.CL 
Link: http://arxiv.org/abs/2310.05421v1 


Título: Poisoned LangChain: Jailbreak LLMs by LangChain 
Categoria: cs.CL 
Link: http://arxiv.org/abs/2406.18122v1 


Título: Breast Ultrasound Report Generation using LangChain 
Categoria: eess.IV 
Link: http://arxiv.org/abs/2312.03013v1 


Título: Revolutionizing Mental Health Care through LangChain: A Journey with a Large Language Model 
Categoria: cs.HC 
Link: http://arxiv.org/abs/2403.05568v1 

=== LLM Response ===
It seems that the tool call yielded a list of research papers related to LangChain, but not specifically focused on education. However, I can try to provide a summary or insights based on the titles and cat

### Usando o Tavily

In [16]:
import os 
from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())

In [17]:
tavily_key = os.environ.get("TAVILY_API_KEY")

In [18]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

In [19]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

search


In [20]:
tavily_tool.search("Me retorne artigos cientificos sobre LangChain", max_results=3)

[Document(id_='885e4985-0b28-46da-b602-cd895707f530', embedding=None, metadata={'url': 'https://python.langchain.com/docs/introduction/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='LangChain simplifies every stage of the LLM application lifecycle: [...] Architecture\u200b\n-----------------------------------------------------------------------------------------------------------\n\nThe LangChain framework consists of multiple open-source libraries. Read more in the Architecture page. [...] Retrieval\n       Retrievers\n       Runnable interface\n       Streaming\n       Structured outputs\n       Testing\n       String-in, string-out llms\n       Text splitters\n       Tokens\n       Tool calling\n       Tools\n       Tracing\n       Vector stores\n       Why LangChain?', path=None, url=None, mimetype=None), image_resource=No

In [21]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker


tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Travily Search",
    description= "Busca artigos com Tavily sobre um determinado tópico"
)

In [22]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls= False,
    llm=llm
)

In [23]:
agent = AgentRunner(agent_worker)
response = agent.chat("me retorne artigos sobre llm e LangChain")

C:\Users\arthu\AppData\Local\Temp\ipykernel_11324\2541408032.py:1: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  agent = AgentRunner(agent_worker)


Added user message to memory: me retorne artigos sobre llm e LangChain
=== Calling Function ===
Calling function: Travily Search with args: {"max_results": 6, "query": "LLM and LangChain"}
=== Function Output ===
[Document(id_='9803f7fb-eec3-42c3-9dc1-cd3015b5b5b1', embedding=None, metadata={'url': 'https://aws.amazon.com/what-is/langchain/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='LangChain is an open source framework for building applications based on large language models (LLMs). LLMs are large deep-learning models pre-trained on large amounts of data that can generate responses to user queries—for example, answering questions or creating images from text-based prompts. LangChain provides tools and abstractions to improve the customization, accuracy, and relevancy of the information the models generate. For example, dev

In [24]:
print(response)

It seems like you've provided a list of documents related to LangChain and LLMs. LangChain is an open-source framework for building applications powered by large language models (LLMs). It provides a standard interface for models, embeddings, vector stores, and more, allowing developers to build LLM-powered applications with ease.

LangChain can be used for various applications, such as chatbots, conversational interfaces, document Q&A, and knowledge retrieval systems. It also integrates seamlessly with other LangChain products, providing a full suite of tools for building LLM applications.

Some of the key benefits of LangChain include:

* Simplifying the development of LLM-powered applications
* Providing a standard interface for models, embeddings, and vector stores
* Allowing for easy integration with external data sources and software workflows
* Enabling the use of multiple LLMs in a single application
* Providing a modular environment for building and testing LLM applications

O

In [25]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [26]:
url = "files/LLM.pdf"
artigo= SimpleDirectoryReader(input_files=[url]).load_data()

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [27]:
url = "files/LLM_2.pdf"
tutorial= SimpleDirectoryReader(input_files=[url]).load_data()

###Gerar os Embeddings

In [28]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [29]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-large"
)

In [30]:
artigo_index= VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

In [31]:
artigo_index.storage_context.persist(persist_dir="artigo")
tutorial_index.storage_context.persist(persist_dir="tutorial")

###Engine de Busca

In [32]:
from llama_index.core import StorageContext, load_index_from_storage

In [33]:
storage_context = StorageContext.from_defaults(
    persist_dir="artigo"
)

artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
    persist_dir="tutorial"
)

tutorial_index = load_index_from_storage(storage_context)

Loading llama_index.core.storage.kvstore.simple_kvstore from artigo\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from artigo\index_store.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from tutorial\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from tutorial\index_store.json.


In [34]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3, llm=llm)
tutorial_engine = tutorial_index.as_query_engine(similarity_top_k=3, llm=llm)

In [35]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [36]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "fornece informações sobre llm e langChain"
                "Use um apergunta detalhada e texto simples como entrada pra ferramenta"
            )
        )
    ), 

QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description=(
                "fornece informações sobre casos de uso e aplicações de llm"
                "Use um apergunta detalhada e texto simples como entrada pra ferramenta"
            )
        )
    )
]



In [37]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

C:\Users\arthu\AppData\Local\Temp\ipykernel_11324\1421250311.py:8: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  agent_document = AgentRunner(agent_worker)


In [38]:
response = agent_document.chat(
    "quais as principais aplicações posso costruir com llm e langchain?"
)

Added user message to memory: quais as principais aplicações posso costruir com llm e langchain?
=== Calling Function ===
Calling function: artigo_engine with args: {"input": "quais as principais aplica\u00e7\u00f5es posso costruir com llm e langchain?"}
=== Function Output ===
Based on the provided context, some of the main applications you can build with LLMs include chatbots and virtual assistants, code generation and debugging, sentiment analysis, text classification and clustering, language translation, summary and paraphrasing, and content generation.
=== LLM Response ===
Based on the provided context, some of the main applications you can build with LLMs include chatbots and virtual assistants, code generation and debugging, sentiment analysis, text classification and clustering, language translation, summary and paraphrasing, and content generation.


In [39]:
print(response)

Based on the provided context, some of the main applications you can build with LLMs include chatbots and virtual assistants, code generation and debugging, sentiment analysis, text classification and clustering, language translation, summary and paraphrasing, and content generation.


###Agente ReAct

In [40]:
from llama_index.core.agent import ReActAgent

In [41]:
agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    llm=llm
)

c:\programacao\automacao\Llama\.venv\Lib\site-packages\llama_index\core\agent\react\base.py:154: DeprecationWarning: Call to deprecated class ReActAgent. (ReActAgent has been rewritten and replaced by llama_index.core.agent.workflow.ReActAgent.

This implementation will be removed in a v0.13.0 and the new implementation will be promoted to the `from llama_index.core.agent import ReActAgent` path.

See the docs for more information: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
c:\programacao\automacao\Llama\.venv\Lib\site-packages\deprecated\classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)


In [42]:
response = agent.chat("quais as principais ferramentas usadas em LangChain")

Added user message to memory: quais as principais ferramentas usadas em LangChain
=== Calling Function ===
Calling function: Travily Search with args: {"max_results": 6, "query": "LangChain"}
=== Function Output ===
[Document(id_='a6b8c3f1-0a1f-465b-8dd0-63591f0829f1', embedding=None, metadata={'url': 'https://en.wikipedia.org/wiki/LangChain'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Image 5Free and open-source software portal\n\nLangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.(\n\nHistory\n-------\n\n[edit] [...] LangChain was launched in October 2022 as